Based on the following video: [How-to Use The Reddit API in Python by James Briggs](https://www.youtube.com/watch?v=FdjVoOf9HN4)
1. Create a reddit application [here](https://www.reddit.com/prefs/apps) to get a client_id and client_secret


# Example workflow

## Step 1: Find a Trending Post
Use various sorting techniques such as hot, trending, controversial to select a post with insightful information. [Example post](https://www.reddit.com/r/mcp/comments/1nculrw/why_are_mcps_needed_for_basic_tools_like/)

## Step 2: Convert the Post Into a YAML Summary
Apply a rule-based system to generate a snapshot of the post. Ex:
- score each comment: score = upvotes + replys * 10
- output the top 10 comments + any parents
- generate the yaml to use as input for the llm

```yaml
post_title: "Why are MCPs needed for basic tools like filesystem access, git etc?"
post_body: "Im puzzled why an MCP (that too written in TS) is required for something as basic as reading, listing files etc. In my experience, the LLM has no problem in doing these tasks without any reliability concerns without any MCP. Likewise for git and even gh."
children:
    - comment: "MCP servers aren't NEEDED for that, but by using it, then you can re-use that functionality across clients vs implementing it each time in the client. That being said, the embedded functionality would likely work faster/better because of the tighter integration. So basically, you pick your sensitivity to effort cost and take the appropriate route."
    - comment: "LLMs can generate commands but can't actually execute them locally. They have no access to your local resources (e.g. files, apps). Simply put, MCP bridges that gap so the LLM can actually interact with your real environment instead of you having to copy paste commands back and forth. Additionally, they serve as the building blocks for agentic workflows."
      children:
      - comment: "this is the right answer, truly. u/rm-rf-rm your fundamental assumption is incorrect - \"LLM ha sno problem in doing these\" is incorrect. LLMs can say \"list directories in ...\" but it then has to be a tool call by the MCP client that performs that. the question maybe then becomes - should capabilities like file system be implemented by MCP clients by default or be provided via MCPs."
      - comment: "u/rm-rf-rm might have eval {llm_response} where llm_respose is something like git log. In that case s/he might be correct that llm + sh love can do that"
    - comment: "Sure but I think the bigger question is why a separate MCP for Git, LS etc. and not just give the ability to run Bash commands"
      children:
        - comment: "One reason is that the MCP server is self-contained, therefore decoupled from the user's environment and OS specifics. As an example, a git command execution via shell would fail if git is not installed or is not in the path. Another is that the MCP layer allows the LLM to operate at the git/version abstraction level not only the shell invocation one, allowing for more precision in terms of selecting the correct tool and its proper usage."
    - comment: "Maybe you are referring to using an AI powered coding tool? Those have AI tools built for reading files, accessing the shell, etc. the power of MCP is you don't need to build a tool that is restricted to a specific AI coding tool, you can just expose the MCP server remotely and any LLM that supports MCP can access it."
    - comment: "They're not needed; most clients have built-in tools for filesystem access. For git, you can just tell the agent to use the \"gh\" command. Same for anything where a command exists."
      children:
        - comment: "For git, you can just tell the agent to use the \"gh\" command gh does not perform most git actions (add, commit, push, etc.) - instead, it is for github-specific actions which aren't normally handled by the 'git' CLI app. Same for anything where a command exists. Agents may be able to use CLI commands such as git, yes, but they work better with the git MCP etc. most clients have built-in tools for filesystem access. You may be correct about this though."
```

## Step 3: Query the LLM:
```yaml
prompt: "generate a short linkedin post from this conversation. Extract the debate or lightbulb moment from this thread and present it."
context: {article_and_comments}
```

## Step 4: Render the Output

**🔧 The Great MCP Debate: Building Blocks vs. Built-ins**

Had a fascinating discussion about why we need Model Context Protocol (MCP) servers for "basic" tasks like file operations and git commands. The lightbulb moment? It's not about what LLMs *can* generate—it's about execution and reusability.

**The core insight:** LLMs can write perfect bash commands, but they can't actually *run* them. That's where MCPs bridge the gap between AI suggestions and real system interaction.

**But here's the real debate:** Should we have specialized MCPs for git, filesystem, etc., or just give LLMs direct shell access?

**The case for specialized MCPs:**
• Abstraction over raw commands (git operations vs. shell invocations)  
• Environment-agnostic (works regardless of OS or installed tools)
• Reusable across different AI clients
• Better error handling and context

**The case for shell access:**
• Simpler implementation
• Leverages existing tooling
• More flexible for edge cases

The thread revealed a fundamental tension in AI tooling: granular, safe abstractions vs. powerful, direct access. Both approaches have merit depending on your use case and risk tolerance.

What's your take? Specialized tools or universal shell access for AI agents?

#AI #MCP #DeveloperTools #LLM #Automation


In [10]:
import requests
import dotenv
import os
import pandas as pd

In [11]:
## Authenticate with the reddit API

dotenv.load_dotenv()
REDDIT_CLIENT_ID = os.getenv("REDDIT_CLIENT_ID")  # Fixed: was getting CLIENT_SECRET
REDDIT_CLIENT_SECRET = os.getenv("REDDIT_CLIENT_SECRET")
REDDIT_USERNAME = os.getenv("REDDIT_USERNAME")
REDDIT_PASSWORD = os.getenv("REDDIT_PASSWORD")

res = requests.post('https://www.reddit.com/api/v1/access_token',
    auth=requests.auth.HTTPBasicAuth(REDDIT_CLIENT_ID, REDDIT_CLIENT_SECRET),
    data={
        'grant_type': 'password',
        'username': REDDIT_USERNAME,
        'password': REDDIT_PASSWORD
    },
    headers={'User-Agent': 'MyAPI/0.0.1'}
)

TOKEN = res.json()['access_token']

headers = {
    **{'User-Agent': 'MyAPI/0.0.1'}, 
    **{'Authorization': f'bearer {TOKEN}'}
}

In [12]:
# Access user information

res = requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

print(f'username: {res.json()['subreddit']['display_name']}')

username: u_Verusauxilium


In [13]:
# Get hot posts
subreddit = 'python'
res = requests.get(f'https://oauth.reddit.com/r/{subreddit}/hot', headers=headers)

posts_data = []
for post in res.json()['data']['children']:
    posts_data.append({
        'subreddit': post['data']['subreddit'],
        'title': post['data']['title'],
        'selftext': post['data']['selftext'],
        'upvote_ratio': post['data']['upvote_ratio'],
        'ups': post['data']['ups'],
        'downs': post['data']['downs'],
        'score': post['data']['score'],
        'date': post['data']['created'],
        'id': post['data']['id'],
        'link': post['data']['permalink']
        
    })

df = pd.DataFrame(posts_data)
df.head()

,subreddit,title,selftext,upvote_ratio,ups,downs,score,date,id,link
0,Python,Sunday Daily Thread: What's everyone working o...,# Weekly Thread: What's Everyone Working On Th...,0.86,10,0,10,1.757808e+09,1ngcnn7,/r/Python/comments/1ngcnn7/sunday_daily_thread...
1,Python,Saturday Daily Thread: Resource Request and Sh...,# Weekly Thread: Resource Request and Sharing ...,1.00,8,0,8,1.757722e+09,1nfiys8,/r/Python/comments/1nfiys8/saturday_daily_thre...
2,Python,Another free Python 3 book - Files and Directo...,"If you are interested, you can click the top l...",0.57,2,0,2,1.757873e+09,1ngy2ha,/r/Python/comments/1ngy2ha/another_free_python...
3,Python,Python Interview Questions: From Basics to Adv...,"The article titled ""Python Interview Questions...",0.50,0,0,0,1.757881e+09,1nh1lj2,/r/Python/comments/1nh1lj2/python_interview_qu...
4,Python,MathFlow: an easy-to-use math library for python,Project Site: [https://github.com/cybergeek194...,0.89,96,0,96,1.757773e+09,1nfyq8o,/r/Python/comments/1nfyq8o/mathflow_an_easytou...


In [14]:
# Retrieve the comments of the first article
# The article is a tree structure with each comment having a children list.

first_article_comments = posts_data[0]['link']
res = requests.get(f'https://oauth.reddit.com/{first_article_comments}', headers=headers)
res.json()


[{'kind': 'Listing',
  'data': {'after': None,
   'dist': 1,
   'modhash': None,
   'geo_filter': '',
   'children': [{'kind': 't3',
     'data': {'approved_at_utc': None,
      'subreddit': 'Python',
      'selftext': "# Weekly Thread: What's Everyone Working On This Week? 🛠️\n\nHello /r/Python! It's time to share what you've been working on! Whether it's a work-in-progress, a completed masterpiece, or just a rough idea, let us know what you're up to!\n\n## How it Works:\n\n1. **Show &amp; Tell**: Share your current projects, completed works, or future ideas.\n2. **Discuss**: Get feedback, find collaborators, or just chat about your project.\n3. **Inspire**: Your project might inspire someone else, just as you might get inspired here.\n\n## Guidelines:\n\n* Feel free to include as many details as you'd like. Code snippets, screenshots, and links are all welcome.\n* Whether it's your job, your hobby, or your passion project, all Python-related work is welcome here.\n\n## Example Shares